# <center><span style="color:purple"> Bellamy 🌸 </span></center>
<hr style="border: 1px solid violet;">

## Description: 

__Bellamy__ is rule-based chat bot. Its goal is to be supportive and empathetic, designed to be your best friend.
Its purpose is to be there for you when you are feeling low, anxious, or simply need someone to talk.


In [1]:
import nltk
nltk.download('punkt') 
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

import random

import json

import time

import os
from datetime import datetime

from difflib import SequenceMatcher

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...


In [2]:
"""Reading intents from file"""
def intents_read(file='intents.json'):
    try:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            if 'intents' in data:
#                 print(f"Found {len(data['intents'])} intents.")
                return data['intents']
            else:
                print('Intents do not exist in this file.')
                return []
    except FileNotFoundError:
        print(f"Error: File '{file}' not found.")
        return []

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return []


intents = intents_read()
for intent in intents:
    print(f"Tag: {intent['tag']}")
    print(f"Patterns: {intent['patterns']}")
    print(f"Responses: {intent['responses']}")
    print()

Tag: greeting
Patterns: ['Hi', 'Hey', 'Hello', 'How are you?', 'Good day', "What's up", 'Yo', 'Hi there']
Responses: ['Hey there, friend 💜', "Hello! I'm here for you.", 'Hi! How are you feeling today?', 'Good to see you again!', 'Hiya — what’s on your mind?']

Tag: goodbye
Patterns: ['bye', 'cya', 'see you later', 'talk to you soon', 'goodbye', 'bye friend']
Responses: ["I'll miss you, {name} 💫", 'Take care of yourself.', 'Goodbye, and remember — you matter 💜', 'See you soon, bestie!', 'You’re never truly alone. See you later, {name}!']

Tag: feeling_sad
Patterns: ["I'm sad", 'feeling down', 'I feel terrible', 'not okay', 'bad day', 'everything sucks']
Responses: ["I'm really sorry you're feeling this way. Want to talk about it?", "It's okay to feel sad sometimes. I'm here with you.", "You're not alone. I'm right here, {name} 💜", 'Do you want to share what happened?', "Take a deep breath. I'm sending you a virtual hug 🤗"]

Tag: panic_timer
Patterns: ['I feel a panic attack', 'Start cal

In [3]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

"""This finction lowers, tokenizes, removes punctuation, and lemmatizes words."""
def preprocess(s):
    s = s.lower()
    s = re.sub(r'[^a-z\s]', '', s)  
    tokens = word_tokenize(s)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized


In [17]:
"""Addition function realises fuzzy matching."""
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

"""This finction compare user input with intents and chooses random answer from this category."""
def match_intent(user_input, intents, phrase_threshold=0.7, word_threshold=0.5):
    user_clean = user_input.lower().strip()
    user_tokens = preprocess(user_input)
    best_intent = None
    best_score = 0
    best_weight = 0

    for intent in intents:
        for pattern in intent['patterns']:
            try:
                if re.fullmatch(pattern.lower(), user_clean):
                    return intent 
            except re.error:
                continue  

    # fuzzy matching
    for intent in intents:
        for pattern in intent['patterns']:
            pattern_tokens = preprocess(pattern)
            pattern_clean = " ".join(pattern_tokens)

            phrase_score = similarity(" ".join(user_tokens), pattern_clean)

            word_matches = sum(
                1 for utok in user_tokens
                for ptok in pattern_tokens
                if similarity(utok, ptok) >= word_threshold
            )

            total_weight = 0.3 * phrase_score + 0.7 * (word_matches / len(pattern_tokens))

            if total_weight > best_weight:
                best_weight = total_weight
                best_score = phrase_score
                best_intent = intent

    if best_intent and (best_score >= phrase_threshold or best_weight >= 0.5):
        return best_intent
    return None

In [5]:
"""Greetings depend on what time is it in your computer."""
def get_greeting():
    hour = datetime.now().hour
    if 5 <= hour < 12:
        return "Good morning!"
    elif 12 <= hour < 18:
        return "Good afternoon!"
    elif 18 <= hour < 23:
        return "Good evening!"
    else:
        return "Good night!"

"""This function randomly chooses answer from file intents.json"""
def generate_answer(intent, memory):
    resp = random.choice(intent['responses'])

    if '{name}' in resp:
        name = memory.get('name', 'honey')
        resp = resp.replace('{name}', name)

    if intent['tag'] == 'greeting':
        greeting = get_greeting()
        resp = f"{greeting} {resp}"

    tone = memory.get('tone', 'friendly')
    emoji = memory.get('emoji', True)
    style = memory.get('style', 'soft')

    if tone == 'formal':
        resp = resp.replace("Hey", "Hello").replace("Hiya", "Greetings")
        resp = resp.replace("bestie", "dear friend")

    if style == 'direct':
        resp = resp.replace("💜", "").replace("🦋", "").replace("💫", "").replace("🤗", "")
    elif style == 'poetic':
        resp += " 🌙✨"

    if not emoji:
        resp = ''.join(char for char in resp if char.isalnum() or char.isspace() or char in ",.?!'\"-")

    return resp.strip()


In [6]:
"""Extract name from chat and further use it."""
def extract_name(user_input):
    name_match = re.search(r"(my name is|i am|call me)\s+(\w+)", user_input, re.IGNORECASE)
    if name_match:
        return name_match.group(2)
    return None

In [7]:
"""Panic attack timer, guide how to breathe when individual is struggling."""
def panic_attack_timer():
    print("Bellamy: We're starting right now. Inhale...")
    for i in range(12):
        time.sleep(5)
        if i % 2 == 0:
            print("Bellamy: Exhale...")
        elif i == 5:
            print("Ballamy: Keep going!")
        else:
            print("Bellamy: Inhale...")
    print("Bellamy: You did the greatest job!")

In [8]:
"""This function creates log session during the conversation."""
def save_log_session(session_log, start_time):
    date_str = start_time.strftime("%Y-%m-%d")
    folder = "sessions"
    os.makedirs(folder, exist_ok=True)
    
    existing = [f for f in os.listdir(folder) if f.startswith(date_str)]
    session_number = len(existing) + 1
    
    filename = f"{date_str}-session-{session_number}"
    filepath = os.path.join(folder, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(f"Session start: {start_time}\n\n")
        for line in session_log:
            f.write(line + "\n")
    
    print(f"Session saved to {filepath}")

In [9]:
"""Change preferences of conversation."""
def set_user_preferences(user_input, memory):
    if "formal mode" in user_input:
        memory["tone"] = "formal"
        return "Formality mode activated."
    elif "friendly mode" in user_input:
        memory["tone"] = "friendly"
        return "Switched to friendly tone!"
    elif "emoji off" in user_input:
        memory["emoji"] = False
        return "Emoji turned off."
    elif "emoji on" in user_input:
        memory["emoji"] = True
        return "Emoji turned on."
    elif "poetic mode" in user_input:
        memory["style"] = "poetic"
        return "Poetic mode activated 🌙"
    elif "direct mode" in user_input:
        memory["style"] = "direct"
        return "Direct mode activated."
    return None

In [10]:
"""The function shows instruction with start."""
def show_instructions():
    print("\n💡 Bellamy — your emotional support chatbot\n")
    print("You can just talk to me like you would to a friend.")
    print("I understand emotions like sadness, anxiety, panic, and more.")
    print("Here are a few things you can try:")
    print("""
🗨️  Talk to me:
    - "I'm sad"
    - "I feel a panic attack"
    - "Tell me something nice"
    - "I feel ugly"
    - "Thanks"

⚙️  Adjust how I respond:
    - /tone friendly       (or 'formal')
    - /emoji true          (or 'false' to turn off emojis)
    - /style soft          (or 'direct', 'poetic')

👤  Tell me your name:
    - "My name is Alex"

❌  Exit the chat:
    - Type 'exit' or 'quit'

📝  Session logs:
    - At the end, you can choose whether to save the chat log.

""")

In [ ]:
def chatbot():
    show_instructions()
    
    print("Are you feeling anxious? Bellamy is ready to talk with you!\n")
    print("Type 'exit' or 'quit' to end the chat.\n")
    
    memory = {
        "tone": "friendly",
        "emoji": True,
        "style": "soft"
    }

    session_log = []
    start_time = datetime.now()
    intents = intents_read()
    
    fallback_intent = next((intent for intent in intents if intent['tag'] == 'fallback'), None)

    while True:
        user_input = input("You: ").strip()
        session_log.append(f"You: {user_input}")

        if user_input.lower() in ['exit', 'quit']:
            session_log.append("Bellamy: Goodbye, take care 💜")
            save = input("Save this session? (yes/no): ").strip().lower()
            if save == 'yes':
                save_log_session(session_log, start_time)
            break

        if user_input.startswith("/tone"):
            _, value = user_input.split(maxsplit=1)
            memory["tone"] = value
            print(f"Bellamy: Tone set to '{value}'")
            continue

        elif user_input.startswith("/emoji"):
            _, value = user_input.split(maxsplit=1)
            memory["emoji"] = value.lower() == "true"
            print(f"Bellamy: Emoji turned {'on' if memory['emoji'] else 'off'}")
            continue

        elif user_input.startswith("/style"):
            _, value = user_input.split(maxsplit=1)
            memory["style"] = value
            print(f"Bellamy: Style set to '{value}'")
            continue

        # Extract name
        name = extract_name(user_input)
        if name:
            memory['name'] = name

        matched = match_intent(user_input, intents)

        if matched:
            if matched['tag'] == 'panic_timer':
                response = generate_answer(matched, memory)
                print("Bellamy:", response)
                session_log.append(f"Bellamy: {response}")
                session_log.append("Bellamy: Starting 1-minute breathing exercise.")
                panic_attack_timer()
                continue
            else:
                response = generate_answer(matched, memory)
        elif fallback_intent:
            response = generate_answer(fallback_intent, memory)
        else:
            response = "I'm here for you, even if I didn’t understand that."

        print("Bellamy:", response)
        session_log.append(f"Bellamy: {response}")
        
chatbot()


💡 Bellamy — your emotional support chatbot

You can just talk to me like you would to a friend.
I understand emotions like sadness, anxiety, panic, and more.
Here are a few things you can try:

🗨️  Talk to me:
    - "I'm sad"
    - "I feel a panic attack"
    - "Tell me something nice"
    - "I feel ugly"
    - "Thanks"

⚙️  Adjust how I respond:
    - /tone friendly       (or 'formal')
    - /emoji true          (or 'false' to turn off emojis)
    - /style soft          (or 'direct', 'poetic')

👤  Tell me your name:
    - "My name is Alex"
    - "Call me Sam"

❌  Exit the chat:
    - Type 'exit' or 'quit'

📝  Session logs:
    - At the end, you can choose whether to save the chat log.


Are you feeling anxious? Bellamy is ready to talk with you!

Type 'exit' or 'quit' to end the chat.

You: Hello!
Bellamy: Good afternoon! Hello! I'm here for you.
You: My name is Alex
Bellamy: Nice to meet you, Alex 💜
You: panic attack
Bellamy: Okay, breathe with me, Alex. I’ll guide you through.
Bell

In [12]:
#test units:
# name +
# panic attack timer + 
# log session +
# responses with mistakes +
# joke + 
# greetings depend on hour +
# settings - styles + 